In [ ]:
import os
import sys

sys.path.append('C:/Users/ellag/multiviewtracks') # edit this
# Now import your modules
import MultiViewTracks as mvt
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Debugging: Check Python version and system paths
print(f"Python Version: {sys.version}")
print("Sys Path Directories:")
for p in sys.path:
    print(p)

# Use absolute path to append to sys.path
project_path = r"C:\Users\ellag\Desktop\ACADEMIC PROJECTS\3D_tracking_pipeline\colmap\data"
sys.path.append(project_path)

print("Updated sys.path:", sys.path[-2:])

# Define absolute paths for model_path and tracks_path
model_path = os.path.join(project_path, 'data', 'sparse')  # Absolute path to the sparse directory
tracks_path = os.path.join(project_path, 'data', 'tracks')  # Absolute path to the tracks directory

assert os.path.exists(model_path), f"Error: Model path does not exist: {model_path}"
assert os.path.exists(tracks_path), f"Error: Tracks path does not exist: {tracks_path}"
print(f"Model path exists: {model_path}")
print(f"Tracks path exists: {tracks_path}")

# Debugging: Check directory contents
print(f"Contents of model path ({model_path}):", os.listdir(model_path))
print(f"Contents of tracks path ({tracks_path}):", os.listdir(tracks_path))

In [ ]:

print("Initializing MultiViewTracks Scene...")

# Initialize the scene with absolute paths
scene = mvt.Scene(model_path=model_path,
                  tracks_path=tracks_path,
                  fisheye=False,
                  verbose=False)

# Get cameras information from the scene
scene.get_cameras()

#Make the camera path a continuous line
scene.interpolate_cameras()

#Triangulate fish points that are observed in both left and right
scene.triangulate_multiview_tracks()
#Calculate reprojection erros
scene.get_reprojection_errors()
#Project trajectory points observed in one view using interpolated depth from triangulated trajectories
scene.project_singleview_tracks()
#Combine triangulated and projected trajectories
scene.get_tracks_3d()

#Select camera IDs
camera_ids = [1, 2]
#Input distance between
world_distance = 1

#Print camera IDs with their view numbers (how many frames)
for camera_id in camera_ids:
    print(scene.cameras[camera_id])

#Scale tracks and retrieve reconstruction errors (per-frame difference of the reconstructed camera positions and the known real world distance)
reconstruction_errors = scene.scale(camera_ids, world_distance)

#Rotate tracks to ensure the z is the depth of the scene (by making x and y of the tracks match the first two principal components of the camera paths)
scene.rotate()

# Convert `tracks_3d` to a DataFrame and save to CSV
tracks_3d_df = pd.DataFrame(scene.tracks_3d)

# Save to CSV
tracks_3d_df.to_csv("tracks_3d_output.csv", index=False)
print("3D tracks saved to tracks_3d_output.csv")

In [ ]:
#Plot multi view triangulation
fig, axes = plt.subplots(3, 1, figsize=(20, 20), sharey=True, sharex=True)
axes[0] = mvt.utils.plot_tracks_2d(scene.tracks_triangulated, ax=axes[0], show=False, size=0.1, style='errors')
axes[1] = mvt.utils.plot_tracks_2d(scene.tracks_projected, ax=axes[1], show=False, size=0.1)
axes[2] = mvt.utils.plot_tracks_2d(scene.tracks_3d, ax=axes[2], show=True, size=0.1);

